In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/데이터전처리

/content/drive/MyDrive/데이터전처리


In [2]:
!pip install transformers

In [52]:
import pandas as pd
import random
import pprint # 보기 쉽게 깔끔한 형태로 출력

from transformers import AutoModel, AutoTokenizer

from torch import nn # 신경망(neural network)생성
import torch.nn.functional as F # 활성화함수, 손실함수를 대체하여 코드를 간결하게 만듬

from sklearn.metrics import accuracy_score, f1_score

In [53]:
data = pd.read_excel("./data/data.xlsx", engine="openpyxl")#.iloc[:100]
# data["label_idx"] = [random.choice([0,1,2,3,4]) for _ in range(100)]
display(data.head(2))
data.shape

,sentence,label_idx
0,네,0
1,네 접시 색깔은 다른데 가격은 똑같아요,1997


(15726, 2)

In [54]:
all_indices = data.index

In [55]:
train_size = int(0.8 * len(all_indices)) # data의 70% 훈련용
val_size = len(all_indices) - train_size # data의 30% 검증용
train_idx, val_idx = all_indices[:int(0.8 * len(all_indices))], all_indices[int(0.8 * len(all_indices)):]

num_classes = max(data["label_idx"].tolist())+1 # data 개수 == 인덱스 최대값 + 1

In [56]:
train_idx, val_idx

(RangeIndex(start=0, stop=12580, step=1),
 RangeIndex(start=12580, stop=15726, step=1))

In [57]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, BertModel
from transformers import get_constant_schedule_with_warmup, get_linear_schedule_with_warmup # 학습 스케줄러를 생성
from tqdm.auto import tqdm # 프로그램 실행 진행사항 확인

# transformers 라이브러리에서 BERT 토크나이저를 임포트합니다.
from transformers import BertTokenizer

# 다국어 지원 BERT 모델을 사용하여 토크나이저 객체를 생성합니다.
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")

- BERT 입력형태
  - [CLS] 토큰 + 첫번째 문장 + [SEP] 토큰 + 두번째 문장
    - 문장의 시작과 끝, 문장 간의 구분을 명확하게 인식 가능
  - [CLS] 토큰
    - "Classification token", 입력 문장의 시작 부분에 추가.
    - 이 토큰의 임베딩은 문장 분류 작업에 사용, 문장 전체의 문맥을 요약하는 역할
  - [SEP] 토큰
    - "Separator token", 두 문장을 구분하는 데 사용
  - 임베딩
    - 텍스트 데이터를 컴퓨터가 이해하고 처리할 수 있는 수치형 벡터로 변환하는 과정 혹은 변환된 벡터


In [58]:
class TextClassificationDatasetBatch(Dataset):
    def __init__(self, df, tokenizer, batch_size=4):
        self.tokenizer = tokenizer # 토크나이저를 클래스 변수로 저장
        self.sentences = df["sentence"].tolist() # "sentence" 열의 데이터를 리스트로 변환하여 저장
        self.labels = df["label_idx"].tolist() # "label_idx" 열의 데이터를 리스트로 변환하여 저장

    def __len__(self): # 데이터 셋의 총 길이를 반환(레이블 개수 반환)
        return len(self.labels)

    def __getitem__(self, idx):
        # to_return = {
        #     "sentence_encoded":self.encodings[idx],
        #     "label":self.labels[idx]
        # }
        return (self.sentences[idx], self.labels[idx]) # 인덱스에 해당하는 문장과 레이블을 튜플로 묶어 반환

In [59]:
train_data = data.iloc[train_idx]
val_data = data.iloc[val_idx]

In [60]:
# train_dataset = TextClassificationDataset(train_data, tokenizer)
# val_dataset = TextClassificationDataset(val_data, tokenizer)
train_dataset = TextClassificationDatasetBatch(train_data, tokenizer)
val_dataset = TextClassificationDatasetBatch(val_data, tokenizer)

In [61]:
train_dataset[0]

('네', 0)

In [62]:
# DataLoader : PyTorch에서 데이터 로드하는데 사용하는 클래스
# 데이터셋을 batch_size만큼 분할하여 로드
# shuffle=True: epoch마다 데이터셋을 무작위로 섞음(과적합 방지)
# shuffle=False: 데이터셋을 섞지않음(검증 데이터는 순서가 결과에 영향을 미치지 않기때문)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [92]:
# 'next', 'iter' 함수를 이용해 DataLoader 객체에서 첫 번째 배치를 가져옴
# next: 반복자의 다음 값을 반환, 모든 값 소진시 'StopIteration'예외 발생
# iter: 객체를 인자로 받아 해당 객체의 반복자(iterator)를 반환
t = next(iter(train_dataloader))

In [93]:
t[0]

('왜동도 배달되나요?',
 '단무지 조금만 더 주실래요?',
 '네',
 '떡볶이 1인분만 주문하고 싶은데 포장이 되나요?',
 '콜라는 사이다로 바꿔주세요',
 '베이컨토마토디럭스 버거 세트 얼마인가요?',
 '8개요',
 '메뉴도 미리 주문해야 하나요',
 '시간은 어느정도 걸릴까요?',
 '후라이드 등 여러 가지 많이 있어요',
 '제 거는 다 돼가요? 기다려야 돼요?',
 '점심 아니면 저녁',
 '그러면 매운 김밥 하나 하구요',
 '그러면 우럭 1.5kg 주시겠어요?',
 '네',
 '네')

In [94]:
t[1]

tensor([ 840,  153,    0, 1671, 1790,  316,  798,  998,  686, 2158,    0, 2048,
         900, 1873,    0,    0])

In [63]:
def collate_bert_tokenizer(samples):
  # print("sample : ", samples)
  sentences, labels = [s[0] for s in samples], [s[1] for s in samples]
  global tokenizer
  return tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=512), labels

In [64]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn = collate_bert_tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn = collate_bert_tokenizer)

- input_ids: 전체 입력 데이터의 token id
- attention_mask: 연산이 필요없는 패딩 토큰에 대해 어텐션을 하지 않도록 마스킹해주는 역할
- token_type_ids: 각 token이 어떤 타입에 속하는지 나타내는 역할

In [ ]:
for i, t_data in enumerate(train_dataloader):
  tokenized = tokenizer(t_data[0], return_tensors="pt", padding=True, truncation=True, max_length=512)
  print(tokenized["input_ids"].shape, model(**tokenized)["pooler_output"].shape)
  # model(**tokenized) == model(input_ids=tokenized["input_ids"], token_type_ids=tokenized["token_type_ids"], attention_mask=tokenized["attention_mask"])

In [ ]:
# tokenized.keys()

In [ ]:
# model(**tokenized)

In [65]:
import torch
import torch.nn as nn


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_classes = max(data["label_idx"].tolist())+1 # 2998
epochs = 50
batch_size = 32
learning_rate = 0.003

In [66]:
# nn.Module을 상속하여 사용자 정의 모델 클래스 생성
class CustomModel(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(CustomModel, self).__init__() # 부모 클래스의 생성자 호출
        # 사전 학습된 BERT 모델을 불러옴
        self.bert = BertModel.from_pretrained(bert_model)
        # 드롭아웃 레이어를 추가(과적합 방지)
        self.dropout = nn.Dropout(0.1)
        # 출력 레이어 추가(BERT 모델의 hidden_size를 입력받고, 출력 크기는 분류할 클래스의 개수)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        # 입력 데이터를 BERT 모델에 통과
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # BERT 모델의 출력 중 pooler_output(문장의 전체적인 의미)을 사용
        pooled_output = outputs.pooler_output
        # 드롭아웃 레이어 통과
        dropout_output = self.dropout(pooled_output)
        # 출력 레이어를 통과시켜 최종 출력을 반환
        return self.fc(dropout_output)

# CustomModel 클래스를 사용하여 모델을 생성합니다. BERT 모델로는 "klue/roberta-small"을,
# 분류할 클래스의 개수는 num_classes(2998)를 사용합니다.
model = CustomModel("klue/roberta-small", num_classes).to(device)


optimizer = AdamW(model.parameters(),lr=5e-5)
# 학습 스케줄러를 생성(학습률을 동적으로 조정)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['encoder.layer.5.output.LayerNorm.weight', 'encoder.layer.1.attention.self.query.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.5.attention.self.key.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.1.attention.self.key.weight', 'encoder.layer.3.output.dense.weight', 'encoder.layer.4.attention.self.key.bias', 'encoder.layer.5.intermediate.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.5.attention.output.dense.bias', 'encoder.layer.1.attention.output.LayerNorm.bias', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.2.attention.self.value

In [ ]:
for i, (_input, _label) in enumerate(train_dataloader):
        # 각 레이블을 원핫 인코딩으로 변환하고 텐서로 변환
        _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()
        print(_label.shape, _input['input_ids'].shape)

In [67]:
loss_f = nn.CrossEntropyLoss()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)
    # 모델을 학습 모드로 설정합니다.
    model.train()
    # 입력 데이터와 레이블 전처리
    total_loss = 0
    for i, (_input, _label) in enumerate(tqdm(train_dataloader, desc="Training")):
        input_ids = torch.tensor(_input['input_ids']).to(device)
        attention_mask = torch.tensor(_input['attention_mask']).to(device)
        _label = torch.tensor(_label).to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_f(outputs, _label)

        loss.backward() # 역전파를 수행합니다.
        optimizer.step() # optimizer를 사용하여 파라미터 업데이트
        optimizer.zero_grad() # 그라디언트 초기화
        scheduler.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Training Loss: {avg_train_loss}")

    # 모델을 평가 모드로 설정합니다.
    model.eval()
    all_predictions = []
    all_labels = []
    total_val_loss = 0

    for i, (_input, _label) in enumerate(tqdm(val_dataloader, desc="Validation")):
        input_ids = torch.tensor(_input['input_ids']).to(device)
        attention_mask = torch.tensor(_input['attention_mask']).to(device)
        _label = torch.tensor(_label).to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_f(outputs, _label)
        total_val_loss += loss.item()

        # 모델의 예측과 실제 레이블을 저장
        # 예측은 가장 높은 확률을 가진 클래스를 선택
        all_predictions.extend( torch.argmax(outputs, dim=1).tolist() )
        all_labels.extend(_label.tolist())

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Validation Loss: {avg_val_loss}")

    # 정확도와 가중치 평균 F! score 계산하여 출력
    print(f"Accuracy: {accuracy_score(all_labels, all_predictions)}")
    print(f'F1_score: {f1_score(all_labels, all_predictions, average="weighted")}')



Epoch 1/50
----------


Training:   0%|          | 0/394 [00:00<?, ?it/s]

<ipython-input-67-aa8443a92562>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(_input['input_ids']).to(device)
<ipython-input-67-aa8443a92562>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(_input['attention_mask']).to(device)


Training Loss: 5.555374068657153


Validation:   0%|          | 0/99 [00:00<?, ?it/s]

...

<ipython-input-67-aa8443a92562>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(_input['input_ids']).to(device)
<ipython-input-67-aa8443a92562>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(_input['attention_mask']).to(device)


Training Loss: 0.038236467161825695


Validation:   0%|          | 0/99 [00:00<?, ?it/s]

<ipython-input-67-aa8443a92562>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(_input['input_ids']).to(device)
<ipython-input-67-aa8443a92562>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(_input['attention_mask']).to(device)


Validation Loss: 3.1257727061859284
Accuracy: 0.6853146853146853
F1_score: 0.6584852282235668


/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3637145079.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3637145079.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_predictions.append( torch.argmax(F.softmax(outputs)).item() )
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3637145079.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_labels.append( torch.argmax(F.softmax(_label)).item() )


In [ ]:
# 학습하기 이전 inference 결과
outputs

In [ ]:
# 학습하기 이전 inference 결과
outputs

In [ ]:
_label

In [ ]:
F.softmax(outputs)

/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3142704713.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(outputs)


tensor([8.3459e-08, 7.8275e-08, 1.0000e+00, 1.9766e-07, 1.6954e-07],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
torch.argmax(F.softmax(outputs)).item()

/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3085399197.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.argmax(F.softmax(outputs)).item()


2

tensor([-3.6564, -3.7143, 11.9799, -2.7231, -2.9375], grad_fn=<ViewBackward0>)